## import

In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, random_split
#LWM을 하기위한 라이브러리 가져오기
import DeepMIMOv3
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
import time


plt . rcParams [ 'figure.figsize' ]  =  [ 12 ,  8 ]  # 기본 플롯 크기 설정

## GPU설정

In [71]:
# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [72]:
import torch
print(torch.version.cuda)                   # 설치된 CUDA 버전 (예: '11.7')
print(torch.backends.cudnn.version())       # cuDNN 버전 (예: 8200)
print("CUDA available:", torch.cuda.is_available())  # True


11.8
90100
CUDA available: True


# DeepMIMOv3 다운

In [73]:
# pip install DeepMIMOv3 umap-learn

## 파라미터 수정

In [74]:
## Load and print the default parameters
# bandwith: 0.05GHz(50MHz 대역폭 사용)
parameters = DeepMIMOv3.default_params()
pprint(parameters)

{'OFDM': {'RX_filter': 0,
          'bandwidth': 0.05,
          'selected_subcarriers': array([0]),
          'subcarriers': 512},
 'OFDM_channels': 1,
 'active_BS': array([1]),
 'bs_antenna': {'FoV': array([360, 180]),
                'radiation_pattern': 'isotropic',
                'rotation': array([0, 0, 0]),
                'shape': array([8, 4]),
                'spacing': 0.5},
 'dataset_folder': './Raytracing_scenarios',
 'dynamic_scenario_scenes': array([1]),
 'enable_BS2BS': 1,
 'enable_doppler': 0,
 'enable_dual_polar': 0,
 'num_paths': 5,
 'scenario': 'O1_60',
 'ue_antenna': {'FoV': array([360, 180]),
                'radiation_pattern': 'isotropic',
                'rotation': array([0, 0, 0]),
                'shape': array([4, 2]),
                'spacing': 0.5},
 'user_rows': array([1]),
 'user_subsampling': 1}


In [75]:
## Change parameters for the setup
# Scenario O1_60 extracted at the dataset_folder
#LWM 동적 시나리오 불러오기
#자신의 LWM 파일 위치 경로 작성
# parameters['dataset_folder'] = r'/content/drive/MyDrive/Colab Notebooks/LWM'
scene = 15 # 장면 수
parameters['dataset_folder'] = r'C:\Users\dlghd\졸업프로젝트\LWM'

# scnario = 02_dyn_3p5 <- 다운받은 파일(동적시나리오)
parameters['scenario'] = 'O2_dyn_3p5'
parameters['dynamic_scenario_scenes'] = np.arange(scene) #scene 0~9

# 각 사용자-기지국 채널에 대해 최대 10개 멀티패스 경로 사용
parameters['num_paths'] = 10

# User rows 1-100
parameters['user_rows'] = np.arange(100)
# User 축소하기
parameters['user_subsampling'] = 0.01

# Activate only the first basestation
parameters['active_BS'] = np.array([1])

parameters['activate_OFDM'] = 1

parameters['OFDM']['bandwidth'] = 0.05 # 50 MHz
parameters['OFDM']['subcarriers'] = 512 # OFDM with 512 subcarriers
parameters['OFDM']['selected_subcarriers'] = np.arange(0, 64, 1)
#parameters['OFDM']['subcarriers_limit'] = 64 # Keep only first 64 subcarriers

parameters['ue_antenna']['shape'] = np.array([1, 1]) # Single antenna
parameters['bs_antenna']['shape'] = np.array([1, 32]) # ULA of 32 elements
#parameters['bs_antenna']['rotation'] = np.array([0, 30, 90]) # ULA of 32 elements
#parameters['ue_antenna']['rotation'] = np.array([[0, 30], [30, 60], [60, 90]]) # ULA of 32 elements
#parameters['ue_antenna']['radiation_pattern'] = 'isotropic'
#parameters['bs_antenna']['radiation_pattern'] = 'halfwave-dipole'

In [76]:
print(parameters)

{'dataset_folder': 'C:\\Users\\dlghd\\졸업프로젝트\\LWM', 'scenario': 'O2_dyn_3p5', 'dynamic_scenario_scenes': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]), 'num_paths': 10, 'active_BS': array([1]), 'user_rows': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]), 'user_subsampling': 0.01, 'bs_antenna': {'shape': array([ 1, 32]), 'spacing': 0.5, 'rotation': array([0, 0, 0]), 'FoV': array([360, 180]), 'radiation_pattern': 'isotropic'}, 'ue_antenna': {'shape': array([1, 1]), 'spacing': 0.5, 'rotation': array([0, 0, 0]), 'FoV': array([360, 180]), 'radiation_pattern': 'isotropic'}, 'enable

## dataset 구축

In [77]:
## dataset 구축 (chunked on‑the‑fly generation)
import time, gc
from tqdm import tqdm

# 0~999 씬 인덱스, 한 번에 50개씩 처리
scene_indices = np.arange(scene)
chunk_size   = 5
all_data     = []

# 씬 묶음(chunk)마다 generate_data 호출
for i in tqdm(range(0, len(scene_indices), chunk_size)):
    chunk = scene_indices[i : i+chunk_size].tolist()
    parameters['dynamic_scenario_scenes'] = chunk

    start = time.time()
    data_chunk = DeepMIMOv3.generate_data(parameters)
    print(f"Scenes {chunk[0]}–{chunk[-1]} generation time: {time.time() - start:.2f}s")

    # 바로 all_data에 합치거나, 디스크에 저장해도 OK
    all_data.extend(data_chunk)

    # 메모리 해제
    del data_chunk
    gc.collect()

# 마지막에 하나의 리스트로 합친 데이터셋
dataset = all_data


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

The following parameters seem unnecessary:
{'activate_OFDM'}

Scene 1/5

Basestation 1

UE-BS Channels



Reading ray-tracing: 100%|████████████████████████████████████████████████████| 69006/69006 [00:00<00:00, 92148.23it/s]

Generating channels: 100%|█████████████████████████████████████████████████████████| 727/727 [00:00<00:00, 2161.59it/s]



BS-BS Channels



Reading ray-tracing: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]

Generating channels: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 333.49it/s]


...


Scene 5/5

Basestation 1

UE-BS Channels



Reading ray-tracing: 100%|███████████████████████████████████████████████████| 69006/69006 [00:00<00:00, 141936.18it/s]

Generating channels: 100%|█████████████████████████████████████████████████████████| 727/727 [00:00<00:00, 3268.00it/s]



BS-BS Channels



Reading ray-tracing: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 999.36it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:28<00:00,  9.66s/it]

Scenes 10–14 generation time: 8.80s


In [78]:
print(parameters['user_rows'])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]


In [79]:
print(parameters)

{'dataset_folder': 'C:\\Users\\dlghd\\졸업프로젝트\\LWM', 'scenario': 'O2_dyn_3p5', 'dynamic_scenario_scenes': [10, 11, 12, 13, 14], 'num_paths': 10, 'active_BS': array([1]), 'user_rows': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]), 'user_subsampling': 0.01, 'bs_antenna': {'shape': array([ 1, 32]), 'spacing': 0.5, 'rotation': array([0, 0, 0]), 'FoV': array([360, 180]), 'radiation_pattern': 'isotropic'}, 'ue_antenna': {'shape': array([1, 1]), 'spacing': 0.5, 'rotation': array([0, 0, 0]), 'FoV': array([360, 180]), 'radiation_pattern': 'isotropic'}, 'enable_doppler': 0, 'enable_dual_polar': 0, 'enable_B

# 사용자 접근 데이터

In [80]:
user_data = dataset[0][0]['user']
print(user_data.keys())

dict_keys(['paths', 'LoS', 'location', 'distance', 'pathloss', 'channel'])


# 사용자 채널 정보 확인

In [81]:
# subcarries = 나눈 각각의 주파수 채널
# Channel = H <- 채널 벡터
# 채널 형태
# (user, UE antenna, Bs antenna, subcarrier)
channel = dataset[0][0]['user']['channel']
print(channel.shape)  

(727, 1, 32, 64)


In [82]:
print(dataset[0][0]['user']['channel'][100])

[[[ 8.57045598e-06+5.5781261e-06j  8.89099283e-06+5.0515800e-06j
    9.17921989e-06+4.5066768e-06j ... -1.02173499e-05-4.1711201e-07j
   -1.02239183e-05+1.9928974e-07j -1.01933329e-05+8.1496728e-07j]
  [ 1.02161603e-05+4.4529790e-07j  1.02244285e-05-1.7108337e-07j
    1.01955429e-05-7.8684292e-07j ... -9.00999748e-06+4.8361312e-06j
   -8.70222630e-06+5.3702393e-06j -8.36283198e-06+5.8848323e-06j]
  [ 9.02330430e-06-4.8112561e-06j  8.71700831e-06-5.3462113e-06j
    8.37903553e-06-5.8617388e-06j ... -5.29921817e-06+8.7456565e-06j
   -4.76262221e-06+9.0490685e-06j -4.20871947e-06+9.3195977e-06j]
  ...
  [-7.00710962e-06-7.4477266e-06j -7.44313866e-06-7.0119827e-06j
   -7.85211978e-06-6.5507575e-06j ...  9.82847632e-06+2.8229874e-06j
    9.98071755e-06+2.2256456e-06j  1.00966881e-05+1.6202162e-06j]
  [-9.82065103e-06-2.8500913e-06j -9.97453935e-06-2.2531719e-06j
   -1.00921798e-05-1.6480645e-06j ...  9.89848286e-06-2.5667589e-06j
    9.72583803e-06-3.1585257e-06j  9.51785023e-06-3.7388147e

# 사용자 위치 정보

In [83]:
location = dataset[0][0]['user']['location']
print(location.shape)      # (사용자 수, 3)
print(location[0:4])         # 첫 번째 사용자의 (x, y, z)

(727, 3)
[[-71.03330231 -15.57629967   1.        ]
 [-68.63330078 -15.57629967   1.        ]
 [-52.83330154 -15.57629967   1.        ]
 [-31.23329926 -15.57629967   1.        ]]


# 경로정보

In [84]:
paths = dataset[0][0]['user']['paths']
#사용자 수
print(len(paths))
# 첫 번째 사용자 경로 정보
print(paths[0])

727
{'num_paths': 2, 'DoD_phi': array([-160.941, -160.941], dtype=float32), 'DoD_theta': array([93.6525, 94.7439], dtype=float32), 'DoA_phi': array([19.0585, 19.0585], dtype=float32), 'DoA_theta': array([86.3475, 94.7439], dtype=float32), 'phase': array([ 143.357, -137.611], dtype=float32), 'ToA': array([2.61886e-07, 2.62253e-07], dtype=float32), 'LoS': array([1., 0.], dtype=float32), 'power': array([7.5363324e-09, 3.2098095e-09], dtype=float32)}


# 기지국 정보

In [85]:
bs_data = dataset[0][0]['basestation']
print(bs_data.keys())


dict_keys(['paths', 'LoS', 'location', 'distance', 'pathloss', 'channel'])


# Scene 및 사용자 수

In [86]:
for i, scene in enumerate(dataset[0]):
    user_locs = scene['user']['location']
    print(f"Scene {i}: {len(user_locs)} users")

Scene 0: 727 users


# 채널 수

In [87]:
len(dataset[0][0]['user']['channel'])

727

In [45]:
print(dataset[0][0]['user']['paths'][0])

{'num_paths': 2, 'DoD_phi': array([-160.941, -160.941], dtype=float32), 'DoD_theta': array([93.6525, 94.7439], dtype=float32), 'DoA_phi': array([19.0585, 19.0585], dtype=float32), 'DoA_theta': array([86.3475, 94.7439], dtype=float32), 'phase': array([ 143.357, -137.611], dtype=float32), 'ToA': array([2.61886e-07, 2.62253e-07], dtype=float32), 'LoS': array([1., 0.], dtype=float32), 'power': array([7.5363324e-09, 3.2098095e-09], dtype=float32)}


In [46]:
scene = dataset[0][0] # scene 0
ue_idx = 0 # 첫 번째 사용자
channel = scene['user']['channel'][ue_idx]
print(channel.shape)

(1, 32, 64)


# channel CIR mat 정보 가져오기

In [47]:
import scipy.io as sio

file_path = r'C:\Users\dlghd\졸업프로젝트\LWM\O2_dyn_3p5\scene_0\O2_dyn_3p5.1.CIR.mat'
mat_data = sio.loadmat(file_path)

# 파일 안의 key 확인
print(mat_data.keys())




dict_keys(['__header__', '__version__', '__globals__', 'CIR_array_full'])


In [48]:
# 일반적으로 CIR key는 'CIR' 또는 'cir' 같은 이름일 가능성 높음
H_cir = mat_data['__header__']  
print(H_cir)

b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Wed Jun 30 11:33:01 2021'


# Time-Prediction 시작
## Time Series 형태로 변환
### 단일사용자 채널 예측

In [49]:
# print(dataset[0][0]['user']['channel'][150][0][3])

count = 0
for h in dataset[0][0]['user']['channel'][100][0]:
#     h = h.squeeze(0)
    h_real = h.real
    h_imag = h.imag
    if np.sum(np.abs(h_real)) ==0:
        count+=1
    elif np.sum(np.abs(h_imag)) == 0:
        count+=1

print("0이 존재하는 채널 개수",count)

0이 존재하는 채널 개수 0


In [50]:
import numpy as np

# 1) (user, ue_port, bs_ant, subc) → (bs_ant, subc) 로 squeeze
H = dataset[0][0]['user']['channel'][100, 0]   # shape: (32, 64), complex

# 2) BS 안테나 인덱스 3의 서브캐리어 벡터 (64,)
print("Antenna #3 subcarriers:", H[3])

# 3) 전체 서브캐리어(32×64) 중 값이 정확히 0인 요소 개수
zero_elements = np.sum(H == 0)
print("0+0j인 서브캐리어 개수:", zero_elements)

# 4) 서브캐리어 전부가 0인 안테나 포트(행) 개수
zero_ports = np.sum(np.all(H == 0, axis=1))
print("완전 0+0j 안테나 포트 개수:", zero_ports)

# 5) 만약 “값이 하나도 0이 아닌” 서브캐리어 요소 개수를 보고 싶다면
nonzero_elements = np.sum(np.abs(H) > 0)
print("0이 아닌 서브캐리어 개수:", nonzero_elements)


Antenna #3 subcarriers: [ 5.3233252e-06-8.73100362e-06j  4.7875683e-06-9.03589535e-06j
  4.2344141e-06-9.30795068e-06j  3.6658719e-06-9.54618190e-06j
  3.0840083e-06-9.74972318e-06j  2.4909377e-06-9.91783418e-06j
  1.8888149e-06-1.00499046e-05j  1.2798286e-06-1.01454543e-05j
  6.6619128e-07-1.02041367e-05j  5.0133124e-08-1.02257372e-05j
 -5.6610725e-07-1.02101776e-05j -1.1802904e-06-1.01575160e-05j
 -1.7901845e-06-1.00679417e-05j -2.3935731e-06-9.94178117e-06j
 -2.9882635e-06-9.77949367e-06j -3.5720950e-06-9.58166765e-06j
 -4.1429457e-06-9.34902255e-06j -4.6987411e-06-9.08240327e-06j
 -5.2374617e-06-8.78277933e-06j -5.7571492e-06-8.45123941e-06j
 -6.2559161e-06-8.08898767e-06j -6.7319493e-06-7.69734197e-06j
 -7.1835188e-06-7.27772431e-06j -7.6089841e-06-6.83165945e-06j
 -8.0067985e-06-6.36076902e-06j -8.3755176e-06-5.86676424e-06j
 -8.7137996e-06-5.35144000e-06j -9.0204167e-06-4.81666848e-06j
 -9.2942537e-06-4.26439374e-06j -9.5343166e-06-3.69662257e-06j
 -9.7397324e-06-3.11541817e-06j

## 결측치 제거 및 dataload

In [52]:
# ─────────────────────────────────────────────
# ❶ IterableDataset: 모든 유저·서브캐리어를 스트리밍
import torch
from torch.utils.data import IterableDataset, DataLoader
import numpy as np

class ChannelSeqDataset(IterableDataset):
    """
    • seq_len 개의 과거 채널 벡터(real 64 + imag 64 → 128) → 다음 시점 벡터 예측
    • 벡터는 평균전력 1 로 power‑normalize 후 반환
    """
    def __init__(self, scenes, seq_len: int = 5, eps: float = 1e-9):
        super().__init__()
        self.scenes   = scenes
        self.seq_len  = seq_len
        self.eps      = eps                        # 0 division 방지용 신호세기의 크기 
        ch0           = scenes[0][0]['user']['channel']
        self.U        = ch0.shape[0]               # 사용자 수
        self.A        = ch0.shape[2]               # 안테나 32
        self.S        = ch0.shape[3]               # 서브캐리어 64
        self.vec_len  = 2 * self.A                 # 64 real + imag
        0
    def _vec(self, scene, u: int, sc: int) -> torch.Tensor:
        """(32,) complex → (64,) float32  +  power norm"""
        h = scene[0]['user']['channel'][u, 0, :, sc]          # (32,)
        v = np.concatenate([h.real, h.imag]).astype(np.float32)
        p = np.mean(v * v) + self.eps                         # 평균 전력: 채널 벡터 h의 각 성분의 진폭 제곱을 합산
        v /= np.sqrt(p)                                       # 정규화
        return torch.from_numpy(v)                            # (64,)

    def __iter__(self):
        T = len(self.scenes)
        for t in range(self.seq_len, T):  # 타깃 시점
            past_scenes = self.scenes[t - self.seq_len : t]
            tgt_scene   = self.scenes[t]
            for u in range(self.U):
                for s in range(self.S):
                    seq = torch.stack([self._vec(ps, u, s) for ps in past_scenes])
                    if not torch.any(seq):        # 전부 0이면 skip
                        continue
                    target = self._vec(tgt_scene, u, s)
                    if not torch.any(target):     # target이 0이면 skip
                        continue
                    yield seq, target             # ← 여기서 masked_pos 제거
                # shapes: (5,64) / (1,) / (64,)
    
    def __len__(self):
         return (len(self.scenes) - self.seq_len) * self.U * self.S
# ─────────────────────────────────────────────
# ❷ 학습·검증 DataLoader train : val = 6 : 4
seq_len      = 5
split_ratio  = 0.6
split_idx    = int(len(dataset) * split_ratio)

train_ds = ChannelSeqDataset(dataset[:split_idx], seq_len=seq_len)
val_ds   = ChannelSeqDataset(dataset[split_idx:], seq_len=seq_len)

# train_ds 순회하면서 feature/target min, max 계산


batch_size   = 32
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False)
# ─────────────────────────────────────────────


In [53]:
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data    import TensorDataset, DataLoader

def build_tensor_dataset(raw_ds, batch_size, scaler_x=None, scaler_y=None):
    """
    raw_ds: Iterable of (seq, mpos, tgt)
    - 여기서는 mpos를 무시하고 seq, tgt만 사용합니다.
    Returns:
      loader: DataLoader over (seq, tgt)
      scaler_x, scaler_y: train 데이터에 fit된 MinMaxScaler
    """
    Xs, Ys = [], []
    for seq, _, tgt in raw_ds:
        Xs.append(seq)      # (seq_len, D)
        Ys.append(tgt)      # (D,)

    # stack to tensors
    X = torch.stack(Xs)    # (N, seq_len, D)
    Y = torch.stack(Ys)    # (N, D)

    # fit/train scaler only once
    N, L, D = X.shape
    X_np = X.numpy().reshape(-1, D)   # (N*L, D)
    Y_np = Y.numpy()                  # (N, D)

    if scaler_x is None:
        scaler_x = MinMaxScaler().fit(X_np)
    if scaler_y is None:
        scaler_y = MinMaxScaler().fit(Y_np)

    X_s = scaler_x.transform(X_np).reshape(N, L, D)
    Y_s = scaler_y.transform(Y_np)

    # back to torch
    X_t = torch.from_numpy(X_s).float()
    Y_t = torch.from_numpy(Y_s).float()

    ds     = TensorDataset(X_t, Y_t)
    loader = DataLoader(ds, batch_size=batch_size, shuffle=True)

    return loader, scaler_x, scaler_y


In [54]:
len(train_ds) #4x727x64

186112

In [55]:
len(val_ds) #1x727x64

46528

In [56]:
# 1) DataLoader 설정 확인
print(train_loader)                # DataLoader 정보 전체
print("batch_size:", train_loader.batch_size)
print("dataset:",   train_loader.dataset)

# 총 샘플 수
print("total samples:", len(train_loader.dataset))
# → (len(scenes) - seq_len) * U * S 와 동일한 값

# 총 배치 수
print("total batches:", len(train_loader))
# → ceil(total_samples / batch_size)


# 3) 첫 번째 배치 내용 확인
first_batch = next(iter(train_loader))
seqs, tgts = first_batch
print("seqs.shape:",   seqs.shape)    # (B, seq_len, vec_len)
print("tgts.shape:",   tgts.shape)    # (B, vec_len)


batch_size: 32
dataset: <__main__.ChannelSeqDataset object at 0x00000232B61108C0>
total samples: 186112
total batches: 5816
seqs.shape: torch.Size([32, 5, 64])
tgts.shape: torch.Size([32, 64])


## 이론적 input_size

In [57]:
# ──────────────────────────────────────────────────
# input_size 계산 및 출력 (수정된 버전)
# ──────────────────────────────────────────────────

# 1) 전체 샘플 수
input_size = len(train_ds)

# 2) 배치 크기와 배치 수
batch_size = 32   # 이미 설정된 값
n_batches  = len(train_loader)

print(f"→ total samples (input_size): {input_size}")
print(f"→ batch size: {batch_size}")
print(f"→ total batches: {n_batches}")

# 3) train/val 분할 비율 확인 (선택)
print(f"→ train samples: {len(train_ds)}")
print(f"→ val   samples: {len(val_ds)}")


→ total samples (input_size): 186112
→ batch size: 32
→ total batches: 5816
→ train samples: 186112
→ val   samples: 46528


# 아래 코드 구조
┌──────────────────────────────────────────────────────────────┐
│ input_ids  (B, seq_len, element_length)  ─┐                 │
│ masked_pos (B, num_mask)                  ├─>  LWM backbone │
│                                           │    (12-층 트랜스포머)  
└────────────────────────────────────────────┘         │
            logits_lm  (B, num_mask, element_length)  │   enc_output (B, seq_len, d_model)
                                                      ▼
                        ┌─[풀링]───────────────┐      ←── feat (B, d_model)
                        │ 첫 토큰(0번) 선택    │
                        │   or 평균/최대 풀링 │
                        └──────────────────────┘
                                      ▼
                       FC 헤드  (d_model → hidden_dim → out_dim)
                                      ▼
                                out (B, out_dim)

# 시각적비유

[패치 프로젝터]──▶[Transformer ×12]──▶[LayerNorm]──┐
                                                  ├─▶ 64-차 벡터 (CLS 또는 풀링) ─▶ MLP ─▶ out                                                
[Positional Embedding]─────────────────────────────┘


In [58]:
import torch
import torch.nn as nn
from torch.nn import TransformerEncoder, TransformerEncoderLayer

# ────────────────────────────────────────────────
# 1) GRUWithHead — GRU 백본 + FC-헤드
# ────────────────────────────────────────────────
class GRUWithHead(nn.Module):
    def __init__(self,
                 feat_dim: int,
                 hidden_size: int = 256,
                 num_layers: int = 2,
                 bidirectional: bool = False,
                 dropout: float = 0.2,
                 hidden_dim: int = 256,
                 out_dim: int    = 64,
                 freeze_backbone: bool = False):
        super().__init__()
        self.backbone = nn.GRU(
            input_size   = feat_dim,
            hidden_size  = hidden_size,
            num_layers   = num_layers,
            batch_first  = True,
            bidirectional= bidirectional,
            dropout      = dropout if num_layers > 1 else 0.0
        )
        if freeze_backbone:
            for p in self.backbone.parameters():
                p.requires_grad = False

        gru_out_dim = hidden_size * (2 if bidirectional else 1)
        self.head = nn.Sequential(
            nn.Linear(gru_out_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, out_dim)
        )

    def forward(self, x):
        out, _ = self.backbone(x)         # (B, seq_len, H)
        feat   = out[:, -1, :]            # 마지막 타임스텝
        return self.head(feat)            # (B, out_dim)


# ────────────────────────────────────────────────
# 2) TransformerWithHead — 소형 Transformer 백본 + FC-헤드
# ────────────────────────────────────────────────
class TransformerWithHead(nn.Module):
    def __init__(self,
                 feat_dim: int,
                 n_heads: int   = 4,
                 dim_ff: int    = 256,
                 n_layers: int  = 2,
                 dropout: float = 0.1,
                 hidden_dim: int = 256,
                 out_dim: int    = 64,
                 freeze_backbone: bool = False):
        super().__init__()
        layer = TransformerEncoderLayer(
            d_model = feat_dim,
            nhead   = n_heads,
            dim_feedforward = dim_ff,
            dropout = dropout
        )
        self.backbone = TransformerEncoder(layer, num_layers=n_layers)
        if freeze_backbone:
            for p in self.backbone.parameters():
                p.requires_grad = False

        self.head = nn.Sequential(
            nn.Linear(feat_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, out_dim)
        )

    def forward(self, x):
        z    = self.backbone(x.transpose(0,1))  # (T, B, F)
        feat = z[-1]                             # 마지막 토큰
        return self.head(feat)                  # (B, out_dim)


# ────────────────────────────────────────────────
# 3) NoPrediction — 마지막 타임스텝 그대로 리턴
# ────────────────────────────────────────────────
class NoPrediction(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x[:, -1, :]           # (B, feat_dim)


# ────────────────────────────────────────────────
# 4) SequentialPAD — AR(1) φ 추정 후 예측
# ────────────────────────────────────────────────
class SequentialPAD(nn.Module):
    def __init__(self, eps: float = 1e-6):
        super().__init__()
        self.eps = eps

    def forward(self, x):
        xt   = x[:, 1:, :]          # (B, T-1, F)
        xtm1 = x[:, :-1, :]         # (B, T-1, F)
        num  = (xt * xtm1).sum(dim=(1,2))
        den  = (xtm1 * xtm1).sum(dim=(1,2)) + self.eps
        phi  = (num/den).view(-1,1,1)              
        return (phi * x[:, -1:, :]).squeeze(1)     # (B, F)


# ────────────────────────────────────────────────
# 5) SequentialPVEC — 주성분 벡터 기반 예측
# ────────────────────────────────────────────────
class SequentialPVEC(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        B, T, F = x.shape
        X_flat  = x.reshape(-1, F)                  
        C       = (X_flat.t() @ X_flat) / (B*T - 1)  # (F, F)
        eigvals, eigvecs = torch.linalg.eigh(C)     # 고윳값 분해
        v1      = eigvecs[:, -1].view(1,1,F).to(x.device)
        coeff   = (x[:, -1:, :] * v1).sum(dim=2, keepdim=True)  
        return (coeff * v1).squeeze(1)                     # (B, F)


# ────────────────────────────────────────────────
# 6) SequentialRNN — RNN 백본 + Linear
# ────────────────────────────────────────────────
class SequentialRNN(nn.Module):
    def __init__(self, feat_dim, hidden=128):
        super().__init__()
        self.rnn  = nn.RNN(feat_dim, hidden, batch_first=True)
        self.head = nn.Linear(hidden, feat_dim)

    def forward(self, x):
        out, _ = self.rnn(x)
        return self.head(out[:, -1])  # (B, feat_dim)


# ────────────────────────────────────────────────
# 7) SequentialLSTM — LSTM 백본 + Linear
# ────────────────────────────────────────────────
class SequentialLSTM(nn.Module):
    def __init__(self, feat_dim, hidden=128):
        super().__init__()
        self.lstm = nn.LSTM(feat_dim, hidden, batch_first=True)
        self.head = nn.Linear(hidden, feat_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.head(out[:, -1])  # (B, feat_dim)


# ────────────────────────────────────────────────
# 8) ParallelTransformer — 병렬 Transformer 백본 + Linear 헤드
# ────────────────────────────────────────────────
class ParallelTransformer(nn.Module):
    def __init__(self, feat_dim, heads: int = 4, dim_ff: int = 256, layers: int = 2):
        super().__init__()
        layer = TransformerEncoderLayer(
            d_model = feat_dim,
            nhead   = heads,
            dim_feedforward = dim_ff
        )
        self.enc  = TransformerEncoder(layer, num_layers=layers)
        self.head = nn.Linear(feat_dim, feat_dim)

    def forward(self, x):
        # x: (B, seq_len, feat_dim)
        z    = self.enc(x.transpose(0,1))  # (T, B, F)
        feat = z[-1]                        # 마지막 토큰
        return self.head(feat)             # (B, feat_dim)

In [59]:
MODEL_CATALOG = {
    "gru": GRUWithHead,
    "transformer": TransformerWithHead,
    "nopred": NoPrediction,
    "pad": SequentialPAD,
    "pvec": SequentialPVEC,
    "seqrnn": SequentialRNN,
    "seqlstm": SequentialLSTM,
    "parallel_transformer": ParallelTransformer,
}

In [60]:
# # model: LWMWithHead 인스턴스
# print("Backbone requires_grad flags:")
# for name, param in model.backbone.named_parameters():
#     print(f"  {name:40s}: {param.requires_grad}")

# # 또한 전체가 동결됐는지 한 줄로 요약하려면:
# all_frozen = all(not p.requires_gㅁrad for p in model.backbone.parameters())
# print(f"\n→ Is backbone fully frozen? {all_frozen}")


In [61]:
# print(train_ds.U)

## input_size
-input_size = (scene - seq_len) * U * S -> (10-5)+69040*64 = 22092800  
-batch_size = 32  
-배치 수 = input_size / batch_size = 690400배치

In [62]:
import torch
import torch.nn as nn
from torch.optim import Adam

# ── 0) 디바이스 ──────────────────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device =", device)

# ── 1) train_loader에서 시퀀스 길이와 피처 차원 추출 ─────────
first_batch = next(iter(train_loader))
seqs, _     = first_batch            # (B, seq_len, feat_dim), (B, feat_dim)
_, seq_len, feat_dim = seqs.shape
out_dim     = feat_dim
hidden_dim  = 256

print(f"seq_len = {seq_len}, feat_dim = {feat_dim}")

# ── 2) 모델 카탈로그 ─────────────────────────
MODEL_CATALOG = {
    "gru":                  GRUWithHead,
    "transformer":          TransformerWithHead,
    "nopred":               NoPrediction,
    "pad":                  SequentialPAD,
    "pvec":                 SequentialPVEC,
    "seqrnn":               SequentialRNN,
    "seqlstm":              SequentialLSTM,
    "parallel_transformer": ParallelTransformer,
}

# ── 3) 실험할 모델 선택 ─────────────────────
model_name = "gru"  # 원하는 모델 이름
ModelCls   = MODEL_CATALOG[model_name]

# ── 4) 모델별 인자 조립 ──────────────────────
if model_name == "gru":
    args = dict(
        feat_dim        = feat_dim,
        hidden_size     = 128,
        num_layers      = 2,
        bidirectional   = False,
        dropout         = 0.2,
        hidden_dim      = hidden_dim,
        out_dim         = out_dim,
        freeze_backbone = False,
    )
elif model_name == "transformer":
    args = dict(
        feat_dim        = feat_dim,
        n_heads         = 4,
        dim_ff          = 256,
        n_layers        = 2,
        dropout         = 0.1,
        hidden_dim      = hidden_dim,
        out_dim         = out_dim,
        freeze_backbone = False,
    )
elif model_name == "nopred":
    args = {}
elif model_name == "pad":
    args = dict(eps=1e-6)
elif model_name == "pvec":
    args = {}
elif model_name == "seqrnn":
    args = dict(feat_dim=feat_dim, hidden=128)
elif model_name == "seqlstm":
    args = dict(feat_dim=feat_dim, hidden=128)
elif model_name == "parallel_transformer":
    args = dict(feat_dim=feat_dim, heads=4, dim_ff=256, layers=2)
else:
    raise ValueError(f"unknown model: {model_name}")

# ── 5) 인스턴스화 & 옵티마이저 ───────────────
model     = ModelCls(**args).to(device)
print(f"🟢 {model_name} initialized with", args)

criterion = nn.MSELoss()
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)


device = cuda
seq_len = 5, feat_dim = 64
🟢 gru initialized with {'feat_dim': 64, 'hidden_size': 128, 'num_layers': 2, 'bidirectional': False, 'dropout': 0.2, 'hidden_dim': 256, 'out_dim': 64, 'freeze_backbone': False}


In [63]:
import torch
import torch.nn as nn

def call_model(model: nn.Module, x: torch.Tensor) -> torch.Tensor:
    """
    모든 모델을 한 줄로 호출하기 위한 헬퍼.
    · 모든 모델의 forward(x) 형태로 통일되어 있습니다.
    """
    return model(x)


In [64]:
# !pip install umap-learn


In [65]:
def train_one_epoch(model, loader, optimizer, device):
    model.train()
    running = 0.0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)

        optimizer.zero_grad()
        pred = model(xb)               # call_model 대신 직접 호출
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()

        running += loss.item() * xb.size(0)
    return running / len(loader.dataset)

@torch.no_grad()
# def evaluate(model, loader, device):
#     model.eval()
#     tot_rmse = tot_nmse = tot_n = 0.0
#     for xb, yb in loader:
#         xb, yb = xb.to(device), yb.to(device)

#         pred = model(xb)               # 동일하게 직접 호출
#         bs   = xb.size(0)

#         tot_rmse += rmse(pred, yb).item() * bs
#         tot_nmse += nmse(pred, yb).item() * bs
#         tot_n    += bs

#     return tot_rmse / tot_n, tot_nmse / tot_n


In [67]:
# 모델 평가 함수
import torch
import torch.nn.functional as F

# ─────────────────────────────────────────
# 1. 배치 단위 RMSE, NMSE 함수
# ─────────────────────────────────────────
def rmse(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """
    Root-Mean-Squared Error
    returns: 스칼라 (배치 평균)
    """
    return torch.sqrt(F.mse_loss(pred, target, reduction="mean"))   # √MSE

def nmse(pred: torch.Tensor, target: torch.Tensor, eps : float = 1e-12) -> torch.Tensor:
    """
    Normalized MSE  =  E[‖ŷ − y‖²] / E[‖y‖²]
      · 채널 예측 분야에서 흔히 쓰는 지표
    returns: 스칼라 (배치 평균)
    """
    # (B, …) → (B,)  : 각 샘플별 제곱합
    mse_per_sample   = ((pred - target)**2).view(pred.size(0), -1).sum(dim=1)
    power_per_sample = (target**2).view(target.size(0), -1).sum(dim=1) + eps
    return (mse_per_sample / power_per_sample).mean()


# ─────────────────────────────────────────
# 2. 검증 루프 예시
# ─────────────────────────────────────────
def evaluate(model, loader, device="cuda"):
    """
    Validation loop for IterableDataset without masked_pos.
    Returns average RMSE and NMSE over all samples.
    """
    model.eval()
    total_rmse, total_nmse, total_samples = 0.0, 0.0, 0

    with torch.no_grad():
        for input_ids, target in loader:
            # Move to device
            input_ids = input_ids.to(device)
            target    = target.to(device)

            # Batch size
            bs = input_ids.size(0)

            # Forward pass
            pred = model(input_ids)

            # Accumulate batch metrics
            total_rmse    += rmse(pred, target).item() * bs
            total_nmse    += nmse(pred, target).item() * bs
            total_samples += bs

    # Compute averages
    return {
        "RMSE": total_rmse / total_samples,
        "NMSE": total_nmse / total_samples
    }



# model training

In [69]:
from tqdm import tqdm
import time, torch
import torch.nn as nn
from torch.optim import Adam

num_epochs = 10
LR         = 1e-4

for model_name, ModelCls in MODEL_CATALOG.items():
    # 1) 모델 인자 조립
    if model_name == "gru":
        args = dict(
            feat_dim        = feat_dim,
            hidden_size     = 128,
            num_layers      = 2,
            bidirectional   = False,
            dropout         = 0.2,
            hidden_dim      = hidden_dim,
            out_dim         = out_dim,
            freeze_backbone = False,
        )
    elif model_name == "transformer":
        args = dict(
            feat_dim        = feat_dim,
            n_heads         = 4,
            dim_ff          = 256,
            n_layers        = 2,
            dropout         = 0.1,
            hidden_dim      = hidden_dim,
            out_dim         = out_dim,
            freeze_backbone = False,
        )
    elif model_name == "nopred":
        args = {}
    elif model_name == "pad":
        args = dict(eps=1e-6)
    elif model_name == "pvec":
        args = {}
    elif model_name == "seqrnn":
        args = dict(feat_dim=feat_dim, hidden=128)
    elif model_name == "seqlstm":
        args = dict(feat_dim=feat_dim, hidden=128)
    elif model_name == "parallel_transformer":
        args = dict(feat_dim=feat_dim, heads=4, dim_ff=256, layers=2)
    else:
        raise ValueError(f"unknown model: {model_name}")

    # 2) 모델 초기화
    model = ModelCls(**args).to(device)

    # 2-1) 파라미터 확인 → 없는 경우 건너뛰기
    params = list(filter(lambda p: p.requires_grad, model.parameters()))
    if len(params) == 0:
        print(f"⚠️  {model_name} has no trainable parameters. Skipping...\n")
        continue

    optimizer = Adam(params, lr=LR)

    print(f"\n▶▶▶ Training {model_name} ◀◀◀")
    start_time = time.time()

    # 3) 에폭별 학습 & 검증
    for epoch in range(1, num_epochs + 1):
        model.train()
        run_loss = 0.0
        tq = tqdm(train_loader, desc=f"[{model_name} {epoch:02d}/{num_epochs}] train", leave=False)
        for b, (xb, yb) in enumerate(tq, 1):
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()
            run_loss += loss.item()
            if b % 100 == 0:
                tq.set_postfix(loss=run_loss / b)
        avg_train_loss = run_loss / b

        # ── VALID ───────────────────────────────
        metrics     = evaluate(model, val_loader, device)
        val_rmse    = metrics["RMSE"]
        val_nmse    = metrics["NMSE"]
        val_nmse_db = 10 * torch.log10(torch.tensor(val_nmse)).item()

        # ── 로그 출력 ───────────────────────────
        print(
            f"[{epoch:02d}/{num_epochs}] "
            f"Train Loss: {avg_train_loss:.4f}  "
            f"Val RMSE: {val_rmse:.4f}  "
            f"Val NMSE: {val_nmse:.4e}  "
            f"Val NMSE_dB: {val_nmse_db:.1f} dB"
        )

    # 4) 총 소요 시간
    total_time = time.time() - start_time
    print(f"{model_name} total training time: {total_time:.2f}s")



▶▶▶ Training gru ◀◀◀


[01/10] Train Loss: 0.0065  Val RMSE: 0.0576  Val NMSE: 1.0909e+00  Val NMSE_dB: 0.4 dB


[02/10] Train Loss: 0.0046  Val RMSE: 0.0562  Val NMSE: 1.0206e+00  Val NMSE_dB: 0.1 dB


[03/10] Train Loss: 0.0043  Val RMSE: 0.0554  Val NMSE: 1.0223e+00  Val NMSE_dB: 0.1 dB


[04/10] Train Loss: 0.0041  Val RMSE: 0.0548  Val NMSE: 1.0583e+00  Val NMSE_dB: 0.2 dB


...

[06/10] Train Loss: 0.0037  Val RMSE: 0.0517  Val NMSE: 1.1456e+00  Val NMSE_dB: 0.6 dB


[07/10] Train Loss: 0.0036  Val RMSE: 0.0513  Val NMSE: 1.1696e+00  Val NMSE_dB: 0.7 dB


[08/10] Train Loss: 0.0035  Val RMSE: 0.0511  Val NMSE: 1.1491e+00  Val NMSE_dB: 0.6 dB


[09/10] Train Loss: 0.0034  Val RMSE: 0.0509  Val NMSE: 1.1534e+00  Val NMSE_dB: 0.6 dB


[10/10] Train Loss: 0.0034  Val RMSE: 0.0506  Val NMSE: 1.1450e+00  Val NMSE_dB: 0.6 dB
parallel_transformer total training time: 21739.94s
